In [1]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
# !pip install tensorflow_text 
!pip install transformers emoji
# !pip install ktrain

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 184 kB 49.7 MB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 
     |████████████████████████████████| 895 kB 33.4 MB/s 
     |████████████████████████████████| 636 kB 42.3 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=8ba5b5c09c979a1b3981450a8e7eea569942b19925ff33996b1b652da84cd70b
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer


In [ ]:
import pandas as pd
dataset = pd.read_excel("/content/drive/MyDrive/English Category Transformer Model/en_category_model_data.xlsx")
dataset = dataset.sample(frac=1, axis=1).sample(frac=1).reset_index(drop=True) 
dataset = dataset[['p_message','Category']]
dataset.head()

,p_message,Category
0,Another invasion of @omantel. Fiber Optic comi...,telecom
1,Collier County tourism numbers show a grim May...,tourism
2,\n@etisalat @Etisalat_Care Dear Team your netw...,telecom
3,I'm a pretty solid 10/12 in this brand. i went...,fashion
4,#OMAN Omantel EGM approves merger of Oman Mobi...,telecom


In [ ]:
dataset.groupby('Category').size()

Category
business         2515
education        2480
entertainment    2473
fashion          2349
food             2427
health           2576
politics         2869
sports           2232
technology       2786
telecom          2825
tourism          1441
transport        2612
weather          2155
dtype: int64

In [ ]:
def to_int_sentiment(label):
  if label == "business":
    return 0
  elif label == "education":
    return 1
  elif label == 'entertainment':
    return 2
  elif label == 'fashion':
    return 3
  elif label == 'food':
    return 4
  elif label == 'health':
    return 5
  elif label == 'politics':
    return 6
  elif label == 'sports':
    return 7
  elif label == 'technology':
    return 8
  elif label == 'telecom':
    return 9
  elif label == 'tourism':
    return 10
  elif label == 'transport':
    return 11
  elif label == 'weather':
    return 12

dataset['Category'] = dataset.Category.apply(to_int_sentiment)
dataset = dataset.dropna()
dataset['Category'] = dataset.Category.apply(int)

In [ ]:
dataset.head()

,p_message,Category
0,Another invasion of @omantel. Fiber Optic comi...,9
1,Collier County tourism numbers show a grim May...,10
2,\n@etisalat @Etisalat_Care Dear Team your netw...,9
3,I'm a pretty solid 10/12 in this brand. i went...,3
4,#OMAN Omantel EGM approves merger of Oman Mobi...,9


In [ ]:
from preprocessing import CleaningText
en_text_clean = CleaningText()



In [ ]:
dataset['p_message'] = dataset['p_message'].apply(str)
dataset['p_message'] = dataset['p_message'].apply(en_text_clean.text_preprocessing)
dataset.head()


,p_message,Category
0,another invasion of fiber optic coming soon to...,9
1,collier county tourism numbers show grim may w...,10
2,dear team your network coverage of local data ...,9
3,im pretty solid 10in this brand went with the ...,3
4,oman omantel egm approves merger of oman mobil...,9


In [ ]:
dataset.groupby('Category').size()

Category
0     2515
1     2480
2     2472
3     2349
4     2427
5     2576
6     2869
7     2232
8     2786
9     2825
10    1441
11    2612
12    2155
dtype: int64

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback


# Read data
# dataset = pd.read_excel("/content/drive/MyDrive/ar_general_sentiment_data.xlsx")
# dataset = dataset[['text','Sentiment']]

# Define pretrained tokenizer and model
model_name = "prajjwal1/bert-small"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=13)

# ----- 1. Preprocess data -----#
# Preprocess data
X = list(dataset["p_message"])
y = list(dataset["Category"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)





Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

In [ ]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='micro')
    precision = precision_score(y_true=labels, y_pred=pred,average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()


***** Running training *****
  Num examples = 25391
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15870


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.940500,0.443422,0.876497,0.876497,0.876497,0.876497
1000,0.427400,0.356330,0.906742,0.906742,0.906742,0.906742
1500,0.376100,0.364438,0.909893,0.909893,0.909893,0.909893
2000,0.345000,0.350433,0.917927,0.917927,0.917927,0.917927
2500,0.327200,0.361743,0.920447,0.920447,0.920447,0.920447
3000,0.341100,0.299238,0.926749,0.926749,0.926749,0.926749
3500,0.212600,0.304784,0.933522,0.933522,0.933522,0.933522
4000,0.182900,0.339774,0.929899,0.929899,0.929899,0.929899
4500,0.194500,0.325777,0.932577,0.932577,0.932577,0.932577
5000,0.178800,0.305635,0.940454,0.940454,0.940454,0.940454


***** Running Evaluation *****
  Num examples = 6348
  Batch size = 8
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 6348
  Batch size = 8
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 6348
  Batch size = 8
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 6348
  Batch size = 8
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Nu

TrainOutput(global_step=15870, training_loss=0.16660035216890948, metrics={'train_runtime': 7229.3314, 'train_samples_per_second': 17.561, 'train_steps_per_second': 2.195, 'total_flos': 5023229338137600.0, 'train_loss': 0.16660035216890948, 'epoch': 5.0})

In [ ]:
model_path = "/content/drive/MyDrive/English Category Transformer Model/en_category_model"
trainer.save_model(model_path)

Saving model checkpoint to /content/drive/MyDrive/English Category Transformer Model/en_category_model
Configuration saved in /content/drive/MyDrive/English Category Transformer Model/en_category_model/config.json
Model weights saved in /content/drive/MyDrive/English Category Transformer Model/en_category_model/pytorch_model.bin


In [ ]:

X_test_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512) 
# Create torch dataset
test_dataset = Dataset(X_test_tokenized) 
# Load trained model
# model_path = "sentiment_model"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=13) 
# Define test trainer
test_trainer = Trainer(model) 
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset) 
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

loading configuration file /content/drive/MyDrive/English Category Transformer Model/en_category_model/config.json
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-small",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL

In [ ]:
y_pred

array([ 7,  8, 12, ...,  9,  6,  5])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
class_names = ["0","1","2","3","4","5","6","7","8","9","10","11","12"]
print(classification_report(y_val, y_pred, target_names=class_names))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91       527
           1       0.94      0.93      0.94       495
           2       0.96      0.93      0.95       526
           3       0.98      0.99      0.98       438
           4       0.98      0.94      0.96       491
           5       0.91      0.95      0.93       522
           6       0.96      0.92      0.94       554
           7       0.97      0.97      0.97       449
           8       0.90      0.92      0.91       537
           9       0.95      0.97      0.96       578
          10       0.96      0.96      0.96       292
          11       0.94      0.95      0.95       523
          12       0.96      0.97      0.96       416

    accuracy                           0.95      6348
   macro avg       0.95      0.95      0.95      6348
weighted avg       0.95      0.95      0.95      6348



In [ ]:
 print(confusion_matrix(y_val, y_pred))

[[480   5   2   1   3   6  11   0   8   3   2   4   2]
 [  5 459   5   1   1   7   1   1   8   5   1   1   0]
 [  2   5 490   3   1   6   2   2   5   3   0   5   2]
 [  0   1   0 434   0   0   0   1   0   0   0   1   1]
 [  3   2   2   0 462  10   0   1   0   2   1   5   3]
 [  1   3   4   3   2 496   1   1   7   1   1   1   1]
 [ 16   4   0   0   1   2 509   2  10   2   2   2   4]
 [  0   1   3   1   0   3   1 436   0   0   0   3   1]
 [  9   5   1   0   1   8   1   1 492  13   0   4   2]
 [  2   0   0   0   1   1   0   0  13 560   0   0   1]
 [  8   0   0   0   0   0   0   0   0   0 279   5   0]
 [  3   1   2   0   0   5   0   3   5   1   3 499   1]
 [  1   0   1   1   0   0   4   2   1   1   1   0 404]]


In [ ]:
df = pd.DataFrame(X_val,columns =['p_message'])
df.head()

,p_message
0,magpies writer online to take your questions a...
1,his important event on information and communi...
2,as the sun rose over the east coast this morni...
3,the snow is already starting to fall in fairfi...
4,how are you liking my outfit my wife it out it...


In [ ]:
df['prediction'] = y_pred

In [ ]:
df.head()

,p_message,prediction
0,magpies writer online to take your questions a...,7
1,his important event on information and communi...,8
2,as the sun rose over the east coast this morni...,12
3,the snow is already starting to fall in fairfi...,12
4,how are you liking my outfit my wife it out it...,12


In [ ]:
df.groupby('prediction').size()

prediction
0     530
1     486
2     510
3     444
4     472
5     544
6     530
7     450
8     549
9     591
10    290
11    530
12    422
dtype: int64

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from transformers import AutoTokenizer
from preprocessing import CleaningText

model_name = "prajjwal1/bert-small"
model_path = "en_category"